In [119]:
import pandas as pd
import numpy as np

In [2]:
df_stores = pd.read_csv("../../../../Desktop/store.csv")
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23702 entries, 0 to 23701
Data columns (total 5 columns):
ENSEIGNE     23701 non-null object
ADRESSE 1    23475 non-null object
ADRESSE 2    3938 non-null object
CP           23473 non-null float64
VILLE        23478 non-null object
dtypes: float64(1), object(4)
memory usage: 925.9+ KB


In [4]:
df_stores.dropna(subset=['ADRESSE 1'], inplace=True)
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23475 entries, 0 to 23700
Data columns (total 5 columns):
ENSEIGNE     23475 non-null object
ADRESSE 1    23475 non-null object
ADRESSE 2    3938 non-null object
CP           23473 non-null float64
VILLE        23475 non-null object
dtypes: float64(1), object(4)
memory usage: 1.1+ MB


In [5]:
df_stores.to_csv("stores.csv", index=False)

## Export index ES to csv

In [82]:
from elasticsearch import Elasticsearch
import csv
import unicodedata

In [83]:
def connect_elasticsearch():
    global client
    _es = None
    _es = Elasticsearch([{'host': '192.168.1.10', 'port': 9200}], http_auth=('aboutgoods', 'Vintage*'))
    if _es.ping():
        print('Connected to ElasticSearch!')
    else:
        print('Cannot connect to ElasticSearch!')
    client = _es
    return client

In [84]:
client = connect_elasticsearch()

Connected to ElasticSearch!


In [120]:
# Replace the following Query with your own Elastic Search Query
res = client.search(index="ids_google_places", body=
	{"query": 
    {
	    "match_all": {}
    }
}, size=7031) 

In [121]:
hits = res['hits']['hits']
sources = [x['_source'] for x in hits]

In [123]:
df = pd.DataFrame(sources)
df.head()

,googleResult,keywords,sign,zipOrCity
0,"{'formatted_address': 'Rue Georges Pompidou, 0...",BOUQUET_LEADER_PRICE_SAINT-QUENTIN,LEADER PRICE SAINT-QUENTIN,BOUQUET
1,"{'formatted_address': 'Route de Ouistreham, 14...",SOLIGNY_LA_TRAPPE_LEADER_PRICE_SAINT_AUBIN_D'A...,LEADER PRICE SAINT AUBIN D'ARQUENAY,SOLIGNY LA TRAPPE
2,"{'formatted_address': '28 Rue Rolland Vico, 14...",SOLIGNY_LA_TRAPPE_LEADER_PRICE_ST_GERMAIN_LA_B...,LEADER PRICE ST GERMAIN LA BLANCHE HERBE,SOLIGNY LA TRAPPE
3,"{'formatted_address': 'Route de Ouistreham, 14...",JOURS_EN_VAUX_LEADER_PRICE_SAINT_AUBIN_D'ARQUENAY,LEADER PRICE SAINT AUBIN D'ARQUENAY,JOURS EN VAUX
4,"{'formatted_address': 'Rue Georges Pompidou, 0...",ST_MAUR_LEADER_PRICE_SAINT-QUENTIN,LEADER PRICE SAINT-QUENTIN,ST MAUR


In [124]:
df_google_results= pd.DataFrame(df['googleResult'].tolist())
df_google_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7031 entries, 0 to 7030
Data columns (total 13 columns):
formatted_address     7031 non-null object
geometry              7031 non-null object
icon                  7031 non-null object
id                    7031 non-null object
name                  7031 non-null object
opening_hours         6162 non-null object
permanently_closed    623 non-null object
photos                5854 non-null object
place_id              7031 non-null object
price_level           27 non-null float64
rating                6266 non-null float64
reference             7031 non-null object
types                 7031 non-null object
dtypes: float64(2), object(11)
memory usage: 714.2+ KB


In [125]:
# keep unique id
df_unique_id= df_google_results.drop_duplicates(subset=['place_id'])
df_unique_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2466 entries, 0 to 7017
Data columns (total 13 columns):
formatted_address     2466 non-null object
geometry              2466 non-null object
icon                  2466 non-null object
id                    2466 non-null object
name                  2466 non-null object
opening_hours         2249 non-null object
permanently_closed    26 non-null object
photos                2147 non-null object
place_id              2466 non-null object
price_level           23 non-null float64
rating                2299 non-null float64
reference             2466 non-null object
types                 2466 non-null object
dtypes: float64(2), object(11)
memory usage: 269.7+ KB


In [126]:
#fill NAN with 0 in "permanently_closed"
values = {'permanently_closed': 0}
df_unique_id.fillna(values, inplace=True)

In [127]:
# remove those who are closed
df_unique_id_open = df_unique_id[df_unique_id['permanently_closed'] != True]
df_unique_id_open.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 0 to 7017
Data columns (total 13 columns):
formatted_address     2440 non-null object
geometry              2440 non-null object
icon                  2440 non-null object
id                    2440 non-null object
name                  2440 non-null object
opening_hours         2249 non-null object
permanently_closed    2440 non-null object
photos                2135 non-null object
place_id              2440 non-null object
price_level           22 non-null float64
rating                2284 non-null float64
reference             2440 non-null object
types                 2440 non-null object
dtypes: float64(2), object(11)
memory usage: 266.9+ KB


In [128]:
df_unique_id_open.head()

,formatted_address,geometry,icon,id,name,opening_hours,permanently_closed,photos,place_id,price_level,rating,reference,types
0,"Rue Georges Pompidou, 02100 Saint-Quentin, France","{'location': {'lat': 49.8623231, 'lng': 3.2905...",https://maps.gstatic.com/mapfiles/place_api/ic...,1d82928e727ef87bc36129031a4d9ba222cccef0,Leader Price,"{'open_now': True, 'weekday_text': []}",0,"[{'height': 3984, 'html_attributions': ['<a hr...",ChIJ-9GIEJsY6EcRbOSwsY-qrKc,NaN,3.6,CmRbAAAAi1Y-wKz4PLGvbgzOH2dHFcp3yQGcZ_4upud4k0...,"[supermarket, grocery_or_supermarket, store, f..."
1,"Route de Ouistreham, 14970 Saint-Aubin-d'Arque...","{'location': {'lat': 49.2633901, 'lng': -0.280...",https://maps.gstatic.com/mapfiles/place_api/ic...,c55865115a5f3c2697923e8b6df27db1a4b58627,Leader Price,"{'open_now': False, 'weekday_text': []}",0,"[{'height': 1920, 'html_attributions': ['<a hr...",ChIJ6X69z-tmCkgRUYdTnpxywgQ,NaN,3.8,CmRbAAAAiVKNM-c2o4Cdrzflw3aMdgH2tB0Wz3hKCNFZPY...,"[supermarket, grocery_or_supermarket, store, f..."
2,"28 Rue Rolland Vico, 14280 Saint-Germain-la-Bl...","{'location': {'lat': 49.1892454, 'lng': -0.410...",https://maps.gstatic.com/mapfiles/place_api/ic...,3fe1fc36c8750bc9a8dd7d68ff239592bb9cb6a8,Leader Price,"{'open_now': False, 'weekday_text': []}",0,"[{'height': 1536, 'html_attributions': ['<a hr...",ChIJ-fhb-jhDCkgRfT_UOhQJvvk,NaN,3.8,CmRbAAAAWnD8mfUMmZtlYZQz04Nln7p_EKCwBmCteEiFT2...,"[supermarket, grocery_or_supermarket, store, f..."
10,"CHEMIN DE LA VOILERIE - PLAN DE CAMPAGNE, 1317...","{'location': {'lat': 43.4159485, 'lng': 5.3509...",https://maps.gstatic.com/mapfiles/place_api/ic...,e043bc5bd0be52d38bf83a33153b3dc4d678d8bd,Leader Price,"{'open_now': True, 'weekday_text': []}",0,"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJm7liGqDryRIR4Q73tgyF87w,NaN,1.0,CmRbAAAAnpGunPs4HeArGLdI_sKCIwpW46E-y6uCn2A9Oc...,"[supermarket, grocery_or_supermarket, store, f..."
13,"Z.A. de la Plaine des Bois, 64300 Biron, France","{'location': {'lat': 43.46765449999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,9a06365f543d023ed4cf79c138897d009805cdbb,Leader Price,"{'open_now': True, 'weekday_text': []}",0,"[{'height': 3328, 'html_attributions': ['<a hr...",ChIJyeGSGafyVg0RvEE5ZXPUNZc,NaN,3.7,CmRbAAAAyHuQ3NO1brxA3MA2lXA2p7UNpqWcQz8no7E3tY...,"[supermarket, grocery_or_supermarket, store, f..."


In [129]:
def rename_longitude(x):
    x['lon'] = x.pop('lng')
    return x

In [130]:
# extract location as "location": {"lat": 0.00, "lon": 0.00}
df_unique_id_open['location'] = df_unique_id_open['geometry'].apply(lambda x: x['location'])
df_unique_id_open['location'] = df_unique_id_open['location'].apply(rename_longitude)
df_unique_id_open['location'].head()

/Users/olga/anaconda3/envs/text_classif/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/olga/anaconda3/envs/text_classif/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0             {'lat': 49.8623231, 'lon': 3.2905382}
1            {'lat': 49.2633901, 'lon': -0.2809238}
2            {'lat': 49.1892454, 'lon': -0.4101137}
10            {'lat': 43.4159485, 'lon': 5.3509117}
13    {'lat': 43.46765449999999, 'lon': -0.7510006}
Name: location, dtype: object

In [131]:
df_unique_id_open.drop(columns=['geometry', 'icon', 'permanently_closed', 'reference', 'opening_hours', 'price_level'], inplace=True)
df_unique_id_open.rename(columns={"types": "google_type", "id":"uid"}, inplace=True)
df_unique_id_open['phone'] = np.nan

/Users/olga/anaconda3/envs/text_classif/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


### new_google_places

In [113]:
# Replace the following Query with your own Elastic Search Query
res_new = client.search(index="ids_new_google_places", body=
	{"query": 
    {
	    "match_all": {}
    }
}, size=1000) 

In [114]:
hits_new = res_new['hits']['hits']
sources_new = [x['_source'] for x in hits_new]

In [115]:
df_new = pd.DataFrame(sources_new)
df_new_google_results= pd.DataFrame(df_new['googleResult'].tolist())
df_new_google_results['phone'] = df_new['phone']
df_new_google_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 14 columns):
formatted_address     120 non-null object
geometry              120 non-null object
icon                  120 non-null object
id                    120 non-null object
name                  120 non-null object
opening_hours         116 non-null object
permanently_closed    3 non-null object
photos                112 non-null object
place_id              120 non-null object
plus_code             1 non-null object
rating                116 non-null float64
reference             120 non-null object
types                 120 non-null object
phone                 48 non-null object
dtypes: float64(1), object(13)
memory usage: 13.2+ KB


In [116]:
# keep unique id
df_unique_id_new= df_new_google_results.drop_duplicates(subset=['place_id'])
#fill NAN with 0 in "permanently_closed"
values = {'permanently_closed': 0}
df_unique_id_new.fillna(values, inplace=True)
# remove those who are closed
df_unique_id_open_new = df_unique_id_new[df_unique_id_new['permanently_closed'] != True]
df_unique_id_open_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 0 to 119
Data columns (total 14 columns):
formatted_address     108 non-null object
geometry              108 non-null object
icon                  108 non-null object
id                    108 non-null object
name                  108 non-null object
opening_hours         107 non-null object
permanently_closed    108 non-null object
photos                103 non-null object
place_id              108 non-null object
plus_code             1 non-null object
rating                107 non-null float64
reference             108 non-null object
types                 108 non-null object
phone                 44 non-null object
dtypes: float64(1), object(13)
memory usage: 12.7+ KB


In [117]:
# extract location as "location": {"lat": 0.00, "lon": 0.00}
df_unique_id_open_new ['location'] = df_unique_id_open_new ['geometry'].apply(lambda x: x['location'])
df_unique_id_open_new ['location'] = df_unique_id_open_new ['location'].apply(rename_longitude)
df_unique_id_open_new ['location'].head()

/Users/olga/anaconda3/envs/text_classif/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/olga/anaconda3/envs/text_classif/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0            {'lat': 43.4971751, 'lon': 4.9770878}
1                {'lat': 47.85161, 'lon': -0.3518}
2            {'lat': 50.6513748, 'lon': 2.9786365}
3             {'lat': 49.6432278, 'lon': 3.258766}
4    {'lat': 47.5001478, 'lon': 7.308362199999999}
Name: location, dtype: object

In [118]:
df_unique_id_open_new.drop(columns=['geometry', 'icon', 'permanently_closed', 'reference', 'opening_hours', 'plus_code'], inplace=True)
df_unique_id_open_new.rename(columns={"types": "google_type", "id":"uid"}, inplace=True)
df_unique_id_open_new.head()

,formatted_address,uid,name,photos,place_id,rating,google_type,phone,location
0,"Route de Fos sur Mer Centre Commercial les, Ch...",159bc724b7feb66ecbf053866f53c402103b21a2,Géant Casino,"[{'height': 2988, 'html_attributions': ['<a hr...",ChIJBxhHbYIcthIRN6b-gFN8wbY,3.6,"[supermarket, grocery_or_supermarket, store, f...",0442411400,"{'lat': 43.4971751, 'lon': 4.9770878}"
1,"Route de Laval, 72300 Sablé-sur-Sarthe, France",e395c7a2227ccb54a923b124c88dd7d432e65023,Super U,"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJD7IYQztHCEgROccwOr1jgx0,3.7,"[supermarket, grocery_or_supermarket, store, f...",0243551010,"{'lat': 47.85161, 'lon': -0.3518}"
2,"130 rue Du Grand But Boîte Postale, 129, 59160...",dfd1ecc7c78d3e972a43e723170a15467b99b72e,Carrefour,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJDThMsVfVwkcRwU5c7RtYjGM,3.4,"[supermarket, bank, car_rental, travel_agency,...",None,"{'lat': 50.6513748, 'lon': 2.9786365}"
3,"Centre Commercial 32 Route Nationale, 02300 Vi...",a00b2de7d9376feae9815ff0c949299b4066adf2,Auchan Viry Noureuil,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJrYEb9LJq6EcRBdRx8OszPoo,3.8,"[supermarket, grocery_or_supermarket, store, f...",0323578260,"{'lat': 49.6432278, 'lon': 3.258766}"
4,"20 Rue de la 1ère Armée, 68480 Ferrette, France",b1a8765ceb80d4ac48b45289d1c8d8d76f50920e,Simply Market Ferrette,"[{'height': 2976, 'html_attributions': ['<a hr...",ChIJM0jsYo7skUcRLe1wLqkuKd4,3.7,"[supermarket, bakery, grocery_or_supermarket, ...",0389403176,"{'lat': 47.5001478, 'lon': 7.308362199999999}"


### Concatenate 2 dataframes of Google

In [132]:
df_merged = pd.concat([df_unique_id_open_new,df_unique_id_open])
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2548 entries, 0 to 7017
Data columns (total 9 columns):
formatted_address    2548 non-null object
google_type          2548 non-null object
location             2548 non-null object
name                 2548 non-null object
phone                44 non-null object
photos               2238 non-null object
place_id             2548 non-null object
rating               2391 non-null float64
uid                  2548 non-null object
dtypes: float64(1), object(8)
memory usage: 199.1+ KB


/Users/olga/anaconda3/envs/text_classif/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [139]:
df_merged['phone'] = df_merged['phone'].astype('str')

In [143]:
type(df_merged['phone'].tolist()[1])

str

In [144]:
df_merged['phone'].tolist()[1]

'0243551010'

In [145]:
df_merged.to_csv("../../cleaned_google_places.csv", index=False)